In [ ]:
import torch.nn as nn

class C3D(nn.Module):
    """
    The C3D network as described in [1].
    """

    def __init__(self):
        super(C3D, self).__init__()

        """
            8개의 3D ConvNet 레이어 (3*3*3, stride=1*1*1)
                각 특징 파악해 feature map 만듬
            5개의 pooling layer(2*2*2, stride=2*2*2)....처음만 1*2*2
                중요한 feature만 남김
            2개의 fc 레이어
                2차원을 1차원으로 평탄화
                이전 레이어의 모든 노드가 연결되었기때문에 fully connected
            softmax 출력 레이어
        """
        
        # 커널 사이즈 = d * k * k, 여기서 k=3으로 고정, d만변화

        self.conv1 = nn.Conv3d(3, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1)) 
        # conv layer 1: 필터수 64, padding으로 크기 작아지지않게
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2)) 
        # 첫번째 pooling에서는 1,2,2로 일단 시간 무시, polling과 stride는 같아야 모든 원소 처리. 이때 크기 감소함

        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1)) # conv layer 2 : 필터수 128
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)) # max pooling으로 입력에서 출력 감소

        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1)) # conv layer 3 : 필터수 256
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1)) # conv layer 2 : 필터수 256
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1)) # conv layer 2 : 필터수 256
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))

        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        # 논문대로 여기서 4096개 출력
        self.fc8 = nn.Linear(4096, 487)
        # 이미지 출력 위해 fc

        self.dropout = nn.Dropout(p=0.5) # overfitting 방지

        self.relu = nn.ReLU()

        self.softmax = nn.Softmax(dim=1) # 0~1의 확률로

    # feedfoward network 쌓기
    def forward(self, x):

        h = self.relu(self.conv1(x))
        h = self.pool1(h)

        h = self.relu(self.conv2(h))
        h = self.pool2(h)

        h = self.relu(self.conv3a(h))
        h = self.relu(self.conv3b(h))
        h = self.pool3(h)

        h = self.relu(self.conv4a(h))
        h = self.relu(self.conv4b(h))
        h = self.pool4(h)

        h = self.relu(self.conv5a(h))
        h = self.relu(self.conv5b(h))
        h = self.pool5(h)

        h = h.view(-1, 8192) # reshape, -1로 적절한 행 생성
        h = self.relu(self.fc6(h))
        h = self.dropout(h)
        h = self.relu(self.fc7(h))
        h = self.dropout(h)

        logits = self.fc8(h)
        probs = self.softmax(logits)

        return probs
